In [1]:
print("hi")

import os
import pandas as pd
import subprocess
import time
import re
import numpy as np

import csv

from scripts import combat_info
from scripts import combat_quick_apply
from scripts import combat_quick_QC


CAMCAN = "./DONNES/CamCAN.md.raw.csv.gz"

models_directory = "ADNI_MODELS"
raw_directory = 'RAW/ADNI'
clean_directory = 'ADNI_CLEAN'
best_directory = 'ADNI_BEST'
metric = "md"
method= "classic"

hi


In [2]:
def get_info(mov_data_file):
    [df,bundles] = combat_info.info(mov_data_file)
    nb_hc = int(re.findall('HC\(n=(\d+)',df["DetailInfos"]["Disease"])[0])
    nb_total = df["DetailInfos"]["Number of Subject"]
    nb_sick = nb_total - nb_hc
    return [nb_total,nb_hc,nb_sick]

In [3]:
def get_bundles(mov_data_file):
    return combat_info.get_bundles(mov_data_file)

In [4]:
def fit(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)
    ###########
    ### fit ###
    ###########
    output_model_filename = (
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    cmd = (
        "scripts/combat_quick_fit.py"
        + " "
        + CAMCAN
        + " "
        + mov_data_file
        + " --out_dir "
        + models_directory
        + " --output_model_filename "
        + output_model_filename
        + " --method "
        + method
        + " --hc "
    )
    subprocess.call(cmd, shell=True)
    return

In [5]:
def apply(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)
    clean_mov_data = (
        clean_directory
        +
        '/'
        +
        mov_data.site.unique()[0]
        + "."
        + metric
        + "."
        + method
        + ".csv.gz"
    )
    model = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    combat_quick_apply.apply(mov_data_file, model, clean_mov_data)

In [6]:
def QC(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)

    model_file = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
    )
    clean_mov_data = (
        clean_directory
        +
        '/'
        +
        mov_data.site.unique()[0]
        + "."
        + metric
        + "."
        + method
        + ".csv.gz"
    )
    return combat_quick_QC.QC(CAMCAN,clean_mov_data, model_file)

In [7]:
def get_csv_res(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)
    model = (
            models_directory
            +
            '/'
            +
            mov_data.site.unique()[0]
            + "."
            + metric
            + "."
            + method
            + ".model.csv"
        )
    return combat_quick_apply.make_best(mov_data_file, model)

In [8]:
# mat =[]
# fields = ["Name","NB Suject","NB HC","NB Sick","Mean Bhattacharrya"]

# for filename in sorted(os.listdir(raw_directory)):
#     f = os.path.join(raw_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         if len(fields)==5:
#             fields = fields + get_bundles(f)

#         print(f)
#         infos = [filename]  
#         infos = infos + get_info(f)
#         fit(f)
#         apply(f)
#         dist = QC(f)
#         infos.append(np.mean(dist))
#         infos = infos + dist
#         mat.append(infos)
#         #print("---------------------------")
# print(mat)


In [9]:
# with open('GFG', 'w') as res:
     
#     # using csv.writer method from CSV package
#     write = csv.writer(res)
     
#     write.writerow(fields)
#     write.writerows(mat)

In [10]:
# header = (
#         "sid,bundle,metric,mean,site,age,sex,handedness,disease"
#     )
# combination = np.empty((9, 0))

# for filename in sorted(os.listdir(best_directory)):
#     f = os.path.join(best_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         site = get_csv_res(f)
#         combination = np.hstack([combination, site])
#         print("---------------------------")

# np.savetxt("DONNES/adni_compilation.csv.gz", combination.T, delimiter=",", header=header, fmt="%s", comments="")


In [11]:
# for filename in sorted(os.listdir(best_directory)):
#     f = os.path.join(best_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         print(f)
#         dir = "ADNI_BEST_VIZUALIZE/" + filename.split('.')[0]
#         cmd = (
#             "combat_visualize_data.py"
#             + " "
#             + f
#             + " "
#             + CAMCAN
#             + " --out_dir "
#             + dir
#             + " --bundles all"
#         )
#         subprocess.call(cmd, shell=True)
#         print("---------------------------")

In [12]:
# Define the file
filename = 'DONNES/adni_compilation.csv.gz'
# Check if it is a file
if os.path.isfile(filename):
    print(filename)
    dir = "ADNI_COMPILATION_VIZUALIZE/"
    cmd = (
        "combat_visualize_data.py"
        + " "
        + filename
        + " "
        + CAMCAN
        + " --out_dir "
        + dir
        + " --bundles all"
    )
    subprocess.call(cmd, shell=True)
    print("---------------------------")

DONNES/adni_compilation.csv.gz


/home/local/USHERBROOKE/davy3001/Documents/COMBAT/Jodoin/Combat_robust/.robust/bin/combat_visualize_data.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  __import__('pkg_resources').require('clinical-combat==1.0.0')


---------------------------
